In [2]:
%pylab inline
from pmesh.pm import ParticleMesh
from gaussianCR.construct import *
from gaussianCR.cosmo import *

Populating the interactive namespace from numpy and matplotlib


In [3]:
# some style of the notebook
plt.rc('xtick', labelsize=15)    
plt.rc('ytick', labelsize=15) 
np.set_printoptions(precision=3,linewidth=150,suppress=True)

##  choose cosmology 

We first need to initialize a Cosmos objec that collects some functions (which is dependent on cosmological parameter) required by gsCR object.


To initialize a Cosmos object, one option is to directly initialize from a ``nbodykit.cosmology`` instance

In [4]:
import nbodykit.cosmology as nbcosmos 

wmap9 = Cosmos(FLRW=True,obj=nbcosmos.WMAP9)

Note that obj can also be an instance from ``astropy.cosmology.FLRW``, if set FLRW==True, we use ``nbodykit.cosmology.LinearPower`` to compute the linear power spectrum for the chosen cosmology, using the analytic Eisenstein&Hu approximation.

---------------
Alternatively, one can also intialize Cosmos obj by setting flag `FLRW = Flase`, and input cosmological parameters and linear power spectrum at z=0 by hand. 
Linear power spectrum can be the output generated from `CAMB`, `CLASS` or from alternative cosmology calculater. Note that it should be consistent with the input cosmological parameters.

In [5]:
data = np.loadtxt('/home/yueying/source/gaussianCR/examples/ps/fdm_m2p5_z0_matterpower.dat')
pk_func = interpolate.interp1d(data[:,0],data[:,1],fill_value='extrapolate')
mycosmos = Cosmos(FLRW=False,H0=69.3,Om0=0.286,Ob0=0.0463,Pk_lin=pk_func)

## gsCR object

Here we initialize ``gsCR`` object with:

     wmap9 : Cosmos object we build above
     Lbox  : Boxsize of the initial condition in Mpc/h
     Nmesh : the grid size to represent the linear density field, in shape of (Nmesh,Nmesh,Nmesh)
     RG    : The size of the Gaussian kernel we use to impose the constraints, in unit of Mpc/h
     xpk   : Position to impose the constraint, in unit of Mpc/h. Default is (0, 0, 0), we can change it later.
     CONS  : The flags to turn on the specific constraints, details see below.

The available options of CONS in ('full','f0','f1','f2','vx','vy','vz','TG'): 
 
-        full      : enable all the 18 constraints at position xpk
-        f0        : constrain the height of the density peak (zeroth order of fG field)
-        f1        : constrain the three 1st order derivatives of fG field at xpk, 
-        f2        : constrain the the compactness,ellipticity and orientation of the peak (2nd order derivatives of fG field) 
-        vx,vy,vz  : constrain the three peculiar velocities of fG field at xpk
-        TG        : constrain the tidal field of fG field at xpk

--------------------------------
--------------------------------

``gsCR.set_c_values()`` method, with all availble arguments:

- nu       : peak height, in unit of $\sigma_0(R_G)$
- xd       : peak compactness, in unit of $\sigma_2(R_G)$
- a12sq    : axial ratio $(a_1/a_2)^2$
- a13sq    : axial ratio $(a_1/a_3)^2$
- a1,b1,p1 : Euler angle to transform to principal axis of mass ellipsoid
- vx,vy,vz : Peculiar velocity of the peak in unit km/s
- epsilon  : Shear magnitude in unit km/s/Mpc
- omega    : Shear angle to distribute the shear magnitude between three axes, $[\pi,2\pi]$
- a2,b2,p2 : Euler angle to transform to principal axis of tidal tensor
- silent   : flag to print the relevant peak parameters
-----

Note that we only need to set arguments corresponding to CONS flag, other will be ignored

**Example 1:** 
We build an ensemble mean field of a density peak in the center of a 20 Mpc/h side box, with the peak height $\nu = 4\sigma_0(R_G)$, compactness $x_d = \sigma_2(R_G)$, axial ratio $(a_1/a_2)^2 = 2$ with long axis align along $x$ axis.

In [33]:
fg = gsCR(wmap9,Lbox=20,Nmesh=128,RG=0.9,xpk=[10,10,10],CONS=['f0','f2'])
print (fg)

  This is a gsCR object: 
  Lbox = 20.0 Mpc/h 
  Nmesh = 128
  RG = 0.9 Mpc/h 
  Sigma0_RG = 1.82, Sigma2_RG = 1.78 
  xpk = [10, 10, 10] 
  CONS = ['f0', 'f2'] 
  xij_tensor_inv = 
 None 


-----
We see that currently the `fg.xij_tensor_inv` is not initialzied yet.
After we specify the CONS flags, we need to build the ($\xi_{ij}^{-1}$) matrix constructed by the constraint kernels H0, H5~H10 corresponding to CONS='f0','f2'.
Note that `build_Xij_inv_matrix()` need to be call whenever we change the CONS flags.

In [34]:
fg.build_Xij_inv_matrix()
print (fg.xij_tensor_inv) # this is the covariance matrix we use to build the ensemble mean field

[[ 0.626  0.375  0.375  0.375 -0.     0.     0.   ]
 [ 0.375  2.122 -0.249 -0.249  0.     0.    -0.   ]
 [ 0.375 -0.249  2.122 -0.249 -0.     0.    -0.   ]
 [ 0.375 -0.249 -0.249  2.122  0.    -0.    -0.   ]
 [-0.     0.    -0.     0.     4.748 -0.     0.   ]
 [ 0.     0.     0.    -0.    -0.     4.748 -0.   ]
 [ 0.    -0.    -0.    -0.     0.    -0.     4.748]]


Apart from the $\xi_{ij}^{-1}$ matrix, we also need to set $c_i$ corresponding to the peak features. 

In [35]:
c_values = fg.set_c_values(nu=4,xd=3,a12sq=2)

Constrain peak parameters: 
f0:  nu = 4.0 $\sigma_0$
f2:  xd = 3.0 $\sigma_2$, a12sq = 2.0, a13sq = 1.0,a1=0.00, b1=0.00, p1=0.00


With $\xi_{ij}^{-1}$ matrix and $c_i$ values ready, we can build the Ensemble mean field with 
$\bar{f}(\mathbf x) = <f(\mathbf x)|\Gamma> = \xi_i(\mathbf x) \xi^{-1}_{ij} c_j$: